# **Model Comparision**

Una vez realizada la limpieza de datos, tenemos que decidir que modelo se ajustaría mejor a estos datos, es decir, cual sería el que mejor rendimiento tendría a la hora de detectar phishing en un enlace.

¿Y cómo podemos hacer esto? La mejor opción es mediante validación cruzada. Vamos a evaluar el rendimiento de cada modelo mediante cross validation y veremos cual es el más adecuado para nuestro proyecto.

Para empezar, importaremos las bibliotecas necesarias:

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import recall_score

Importamos nuestro dataframe:

In [4]:
data = pd.read_csv('../data/raw/dataset_phishing.csv')
data.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


Vamos a cambiar la columna categórica 'status' por 1's y 0's para hacerla numérica:

In [5]:
data['status'] = data['status'].map({'phishing': 1, 'legitimate': 0})
data.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,0
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,1
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,1
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,0
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,0


Retiramos la columna de las url, ya que no nos aporta información útil.

In [6]:
data = data.drop('url', axis=1)
data.head()

,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,37,19,0,3,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,0
1,77,23,1,1,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,1
2,126,50,1,4,1,0,1,2,0,3,...,1,0,0,14,4004,5828815,0,1,0,1
3,18,11,0,2,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,0
4,55,15,0,2,2,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,0


Bien, tenemos 88 columnas de dataframe para la X y 1 columna, la de 'status' como y. Obviamente, son demasiadas. ¿Pero como podemos hacer para descartar y ver cuales son las más importantes para quedarnos con ellas? Haciendo Feature Importance. 

### Feature importance

- Lo primero que debemos hacer es separar los datos en X e y. Utilizaremos como X todas las columnas excepto la columna 'status', que será nuestro vector y. Tenemos que dividir, antes que nada, en X_test, X_train, y_test, y_train. ¿Por qué hacemos esto? Porque debemos hacer el feature importance con los datos de train solamente, así evitaremos la filtración de datos.

In [7]:
X = data.iloc[:, 1:-1]
y = data['status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- Una vez dividido, creamos un modelo de árbol de decisión, ajustamos con un fit al conjunto X e y, y utilizamos model.feature_importances_

In [8]:
rnd_clf = RandomForestClassifier(n_estimators = 100, random_state=42)
rnd_clf.fit(X_train, y_train)

importances = rnd_clf.feature_importances_

- Sacamos una lista de tuplas con las importancia de las columnas y el nombre de las columnas.

In [9]:
importances_orden = sorted(zip(importances, data.iloc[:, 1:-1].columns), reverse=True)
importances_orden

[(0.1873389120664313, 'google_index'),
 (0.10634442459361289, 'page_rank'),
 (0.07931349512081924, 'nb_hyperlinks'),
 (0.06201245211768272, 'web_traffic'),
 (0.045555406153859807, 'nb_www'),
 (0.03597322491758546, 'domain_age'),
 (0.03185324166622902, 'ratio_extHyperlinks'),
 (0.029516873856723143, 'longest_word_path'),
 (0.02593758805070382, 'phish_hints'),
 (0.02555639806201825, 'ratio_intHyperlinks'),
 (0.02098320697734806, 'safe_anchor'),
 (0.019903095239908755, 'ratio_digits_url'),
 (0.018948947804339877, 'longest_words_raw'),
 (0.01829187698874836, 'length_hostname'),
 (0.015200280918982066, 'links_in_tags'),
 (0.014951989500831238, 'avg_word_path'),
 (0.014945407175725605, 'length_words_raw'),
 (0.014278144830923727, 'char_repeat'),
 (0.013630785769358146, 'domain_registration_length'),
 (0.013130071897662183, 'ratio_extRedirection'),
 (0.01266690270893229, 'shortest_word_host'),
 (0.012280789924412576, 'ratio_digits_host'),
 (0.012242284784400669, 'nb_slash'),
 (0.0118744579365

- Creamos un dataframe a partir de esta lista de tuplas y eliminamos aquellos que tengan un valor de importancia por debajo de 0,01, pues solo nos quedaremos con las que esten por encima de ese valor de importancia.

In [10]:
df = pd.DataFrame(importances_orden, columns=['Importance', 'column'])
df = df.loc[df['Importance'] >= 0.01]
df['Significado'] = ['Indica si la página web aparece en el índice de Google.',
                    'Ranking de la página web.',
                    'Número de hipervínculos en la página web.',
                    'Tráfico de la página web.',
                    'Indica si la URL contiene "www" en el nombre de host.',
                    'Antigüedad del dominio.',
                    'Proporción de hipervínculos externos en la página web.',
                    'Longitud de la palabra más larga en la ruta de la URL sin caracteres especiales.',
                    'Indica si la URL contiene palabras clave asociadas con phishing.',
                    'Proporción de hipervínculos internos en la página web.',
                    'Indica si los enlaces de la página web utilizan atributos "rel" para evitar la apertura de nuevas pestañas.',
                    'Proporción de dígitos en la URL.',
                    'Longitud de la palabra más larga en la URL sin caracteres especiales.',
                    'Longitud del nombre de host en la URL.',
                    'Indica si hay enlaces en las etiquetas HTML de la página web.',
                    'Longitud promedio de las palabras en la ruta de la URL sin caracteres especiales.',
                    'Longitud de la URL sin caracteres especiales.',
                    'Indica si hay caracteres repetidos en la URL.',
                    'Longitud del registro del dominio.',
                    'Proporción de redirecciones externas en la página web.',
                    'Longitud de la palabra más corta en el nombre de host de la URL sin caracteres especiales.',
                    'Proporción de dígitos en el nombre de host de la URL.',
                    'Número de barras diagonales en la URL.',
                    'Longitud de la palabra más corta en la ruta de la URL sin caracteres especiales.',
                    'Indica si el nombre de dominio está incluido en el título de la página web.',
                    'Número de puntos en la URL.',
                    'Número de guiones en la URL',
                    'Longitud promedio de las palabras en la URL sin caracteres especiales.']
df

,Importance,column,Significado
0,0.187339,google_index,Indica si la página web aparece en el índice d...
1,0.106344,page_rank,Ranking de la página web.
2,0.079313,nb_hyperlinks,Número de hipervínculos en la página web.
3,0.062012,web_traffic,Tráfico de la página web.
4,0.045555,nb_www,"Indica si la URL contiene ""www"" en el nombre d..."
5,0.035973,domain_age,Antigüedad del dominio.
6,0.031853,ratio_extHyperlinks,Proporción de hipervínculos externos en la pág...
7,0.029517,longest_word_path,Longitud de la palabra más larga en la ruta de...
8,0.025938,phish_hints,Indica si la URL contiene palabras clave asoci...
9,0.025556,ratio_intHyperlinks,Proporción de hipervínculos internos en la pág...


Por orden de importancia, las filas más importantes (valor de importance > 0,01) son las siguientes:

    - 0: 'google_index': Indica si la página web aparece en el índice de Google.
    - 1: 'page_rank': Ranking de la página web.
    - 2: 'nb_hyperlinks': Número de hipervínculos en la página web.
    - 3: 'web_traffic': Tráfico de la página web.
    - 4: 'nb_www': Indica si la URL contiene "www" en el nombre de host.
    - 5: 'domain_age': Antigüedad del dominio.
    - 6: 'ratio_extHyperlinks': Proporción de hipervínculos externos en la página web.
    - 7: 'longest_word_path': Longitud de la palabra más larga en la ruta de la URL sin caracteres especiales.
    - 8: 'phish_hints': Indica si la URL contiene palabras clave asociadas con phishing.
    - 9: 'ratio_intHyperlinks': Proporción de hipervínculos internos en la página web.
    - 10: 'safe_anchor': Indica si los enlaces de la página web utilizan atributos "rel" para evitar la apertura de nuevas pestañas.
    - 11: 'ratio_digits_url': Proporción de dígitos en la URL.
    - 12: 'longest_words_raw': Longitud de la palabra más larga en la URL sin caracteres especiales.
    - 13: 'length_hostname': Longitud del nombre de host en la URL.
    - 14: 'links_in_tags': Indica si hay enlaces en las etiquetas HTML de la página web.
    - 15: 'avg_word_path': Longitud promedio de las palabras en la ruta de la URL sin caracteres especiales.
    - 16: 'length_words_raw': Longitud de la URL sin caracteres especiales.
    - 17: 'char_repeat': Indica si hay caracteres repetidos en la URL.
    - 18: 'domain_registration_length': Longitud del registro del dominio.
    - 19: 'ratio_extRedirection': Proporción de redirecciones externas en la página web.
    - 20: 'shortest_word_host': Longitud de la palabra más corta en el nombre de host de la URL sin caracteres especiales.
    - 21: 'ratio_digits_host': Proporción de dígitos en el nombre de host de la URL.
    - 22: 'nb_slash': Número de barras diagonales en la URL.
    - 23: 'shortest_word_path': Longitud de la palabra más corta en la ruta de la URL sin caracteres especiales.
    - 24: 'domain_in_title': Indica si el nombre de dominio está incluido en el título de la página web.
    - 25: 'nb_dots': Número de puntos en la URL.
    - 26: 'nb_hyphens': número de guiones en la URL
    - 27: 'avg_words_raw': Longitud promedio de las palabras en la URL sin caracteres especiales.

- Ahora sacamos una lista con las columnas importantes, las que utilizaremos para que el modelo aprenda.

- MUY IMPORTANTE añadir a la lista la columna de 'status', que será nuestra columna 'y' para el modelo. (Si no la añadimos, cuando hagamos la máscara tambien desaparecerá).

In [11]:
columnas_importantes = df['column'].tolist()
columnas_importantes.append('status')
columnas_importantes

['google_index',
 'page_rank',
 'nb_hyperlinks',
 'web_traffic',
 'nb_www',
 'domain_age',
 'ratio_extHyperlinks',
 'longest_word_path',
 'phish_hints',
 'ratio_intHyperlinks',
 'safe_anchor',
 'ratio_digits_url',
 'longest_words_raw',
 'length_hostname',
 'links_in_tags',
 'avg_word_path',
 'length_words_raw',
 'char_repeat',
 'domain_registration_length',
 'ratio_extRedirection',
 'shortest_word_host',
 'ratio_digits_host',
 'nb_slash',
 'shortest_word_path',
 'domain_in_title',
 'nb_dots',
 'nb_hyphens',
 'avg_words_raw',
 'status']

Ahora hacemos que el dataframe original se quede solo con las columnas importantes, las que tienen un feature importance por encima de 0,01. Con este dataframe trabajaremos.

In [12]:
data_real = list(set(data.columns).intersection(set(columnas_importantes)))
data = data[data_real]
data

,google_index,domain_registration_length,page_rank,safe_anchor,nb_www,ratio_extRedirection,avg_words_raw,ratio_digits_url,nb_dots,ratio_digits_host,...,avg_word_path,length_words_raw,status,phish_hints,shortest_word_path,nb_hyphens,web_traffic,char_repeat,links_in_tags,nb_slash
0,1,45,4,0.000000,1,0.875000,5.750000,0.000000,3,0.000000,...,4.500000,4,0,0,3,0,0,4,80.000000,3
1,1,77,2,100.000000,0,0.000000,15.750000,0.220779,1,0.000000,...,14.666667,4,1,0,2,0,0,4,100.000000,5
2,1,14,0,100.000000,0,0.000000,8.250000,0.150794,4,0.000000,...,8.142857,12,1,0,2,1,5828815,2,100.000000,5
3,0,62,3,62.500000,0,0.250000,5.000000,0.000000,2,0.000000,...,0.000000,1,0,0,0,0,107721,0,100.000000,2
4,0,224,6,0.000000,1,0.537037,6.333333,0.000000,2,0.000000,...,7.000000,6,0,0,4,2,8725,3,76.470588,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,0,448,6,0.000000,1,0.043478,7.750000,0.000000,2,0.000000,...,9.500000,4,0,0,8,0,3980,4,80.000000,4
11426,1,211,0,0.000000,1,0.000000,5.166667,0.023810,5,0.000000,...,4.900000,12,1,0,3,0,0,3,100.000000,5
11427,1,2809,10,80.000000,1,0.000000,6.153846,0.142857,2,0.000000,...,6.272727,13,0,0,1,6,8,5,6.250000,5
11428,0,85,4,0.000000,1,0.050000,12.500000,0.000000,2,0.000000,...,0.000000,2,0,0,0,0,2455493,3,16.666667,3


- Colocamos la columna 'status' la última del dataframe.

In [13]:
col = data.pop('status')
data.insert(len(data.columns), 'status', col)
data.head()

,google_index,domain_registration_length,page_rank,safe_anchor,nb_www,ratio_extRedirection,avg_words_raw,ratio_digits_url,nb_dots,ratio_digits_host,...,avg_word_path,length_words_raw,phish_hints,shortest_word_path,nb_hyphens,web_traffic,char_repeat,links_in_tags,nb_slash,status
0,1,45,4,0.0,1,0.875000,5.750000,0.000000,3,0.0,...,4.500000,4,0,3,0,0,4,80.000000,3,0
1,1,77,2,100.0,0,0.000000,15.750000,0.220779,1,0.0,...,14.666667,4,0,2,0,0,4,100.000000,5,1
2,1,14,0,100.0,0,0.000000,8.250000,0.150794,4,0.0,...,8.142857,12,0,2,1,5828815,2,100.000000,5,1
3,0,62,3,62.5,0,0.250000,5.000000,0.000000,2,0.0,...,0.000000,1,0,0,0,107721,0,100.000000,2,0
4,0,224,6,0.0,1,0.537037,6.333333,0.000000,2,0.0,...,7.000000,6,0,4,2,8725,3,76.470588,5,0


### Ahora ya podemos empezar con la comparativa de modelos.

Separamos los datos en X e y del data nuevo ya realizado el feature importance. Utilizaremos como X todas las columnas excepto la columna 'status', que será nuestro vector y.

In [14]:
X = data.iloc[:, 1:-1]
y = data['status']

Bien, pues una vez dividido el dataframe en X e y, vamos a crear los conjuntos de entrenamiento, X_train e y_train; y los conjuntos de prueba, X_test e y_test, para poder empezar a hacer la comparativa de modelos mediante Cross Validation. Utilizaremos un 20% de los datos para los conjuntos de prueba.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Vamos a hacer una lista a la que llamaremos 'modelos' donde meteremos todos los modelos que vamos a comparar, para posteriormente, mediante un bucle, crear un dataframe de comparación que tenga el modelo y el rendimiento de dicho modelo con estos datos. Primeramente vamos a declarar variables a los distintos tipos de BaggingClassifier para poder diferenciarlos.

In [16]:
BaggingClassifier_LR = BaggingClassifier(base_estimator=LogisticRegression(max_iter=1000))
BaggingClassifier_KNN = BaggingClassifier(base_estimator=KNeighborsClassifier())
BaggingClassifier_SVC = BaggingClassifier(base_estimator=SVC())

In [17]:
modelos = [LogisticRegression(max_iter=1000), KNeighborsClassifier(), DecisionTreeClassifier(), SVC(),
           BaggingClassifier_LR, BaggingClassifier_KNN, BaggingClassifier_SVC, RandomForestClassifier(), AdaBoostClassifier(),
           GradientBoostingClassifier(), XGBClassifier()]

Tenemos diferentes métricas de evaluación que podemos utilizar: accuraccy, recall, f1-score, area bajo la curva ROC. En nuestro caso, debemos tener en cuenta que NO queremos que se nos escape ninguna url que sea phishing, aunque coja como phishing una url legitima. Pero lo que nos interesa es que ningúna url falsa se nos escape. Para eso, nuestra métrica utilizada será el recall.

Creamos el dataframe de comparación 'model_coparision' con 3 columnas: 'Modelo', 'Métrica' y 'Rendimiento'.

In [18]:
model_comparision = pd.DataFrame(columns=['Modelo', 'Metrica', 'Rendimiento Medio'])
model_comparision

,Modelo,Metrica,Rendimiento Medio


Ahora crearemos un bucle 'for' en el que recorreremos la lista de modelos, haciendo validación cruzada y rellenando el dataframe de comparación anteriormente creado.

In [19]:
for i, model in enumerate(modelos):
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
    mean_score = scores.mean()
    model_comparision.loc[len(model_comparision)] = [model, 'recall', mean_score]

model_comparision

,Modelo,Metrica,Rendimiento Medio
0,LogisticRegression(max_iter=1000),recall,0.652660
1,KNeighborsClassifier(),recall,0.846492
2,DecisionTreeClassifier(),recall,0.925860
3,SVC(),recall,0.885740
4,BaggingClassifier(base_estimator=LogisticRegre...,recall,0.800263
5,BaggingClassifier(base_estimator=KNeighborsCla...,recall,0.846711
6,BaggingClassifier(base_estimator=SVC()),recall,0.885304
7,RandomForestClassifier(),recall,0.960967
8,AdaBoostClassifier(),recall,0.927605
9,GradientBoostingClassifier(),recall,0.947450


In [20]:
model_comparision.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Modelo             11 non-null     object 
 1   Metrica            11 non-null     object 
 2   Rendimiento Medio  11 non-null     float64
dtypes: float64(1), object(2)
memory usage: 352.0+ bytes


Elegimos el modelo con el mejor rendimiento según el recall:

In [21]:
mejor_modelo_index = np.argmax(model_comparision['Rendimiento Medio'])
mejor_modelo = model_comparision['Modelo'].iloc[mejor_modelo_index]
print('El índice del mejor modelo para estos datos es:', mejor_modelo_index)
print('El mejor modelo a utilizar para estos datos es:', mejor_modelo)

El índice del mejor modelo para estos datos es: 10
El mejor modelo a utilizar para estos datos es: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...)


Ahora vamos a ajustar el modelo elegido (RandomForestClassifier()) en el conjunto de entrenamiento, X_train e y_train, y vamos a hallar la predicción para poder tener el score del conjunto de prueba:

In [22]:
# Ajustar el modelo elegido en el conjunto de entrenamiento
modelo_elegido = XGBClassifier()
modelo_elegido.fit(X_train, y_train)
y_pred = modelo_elegido.predict(X_test)

Aplicamos el modelo que tiene más rendimiento, el XGBClassiffier() y lo entrenamos con el conjunto de entrenamiento para posteriormente predecir con X_test. Posteriormente, calcularemos la score del area bajo curva ROC.

In [23]:
modelo_elegido = XGBClassifier()
modelo_elegido.fit(X_train, y_train)
y_pred = modelo_elegido.predict(X_test)

test_score = recall_score(y_test, y_pred)
test_score

0.9619131975199291

Por tanto, podemos concluir que:

- La métrica que mejor se adapta a los datos es el recall por la naturaleza de los datos y el resultado que queremos obtener.
- El modelo que mejor rendimiento ha obtenido es el modelo XGBClassifier(), con un rendimiento de 96.62%.
- El score del recall del modelo en el conjunto de prueba es de 96.81%.

In [29]:
#data.to_csv('../data/processed/data_preparada.csv', index=False)
data

,google_index,domain_registration_length,page_rank,safe_anchor,nb_www,ratio_extRedirection,avg_words_raw,ratio_digits_url,nb_dots,ratio_digits_host,...,avg_word_path,length_words_raw,phish_hints,shortest_word_path,nb_hyphens,web_traffic,char_repeat,links_in_tags,nb_slash,status
0,1,45,4,0.000000,1,0.875000,5.750000,0.000000,3,0.000000,...,4.500000,4,0,3,0,0,4,80.000000,3,0
1,1,77,2,100.000000,0,0.000000,15.750000,0.220779,1,0.000000,...,14.666667,4,0,2,0,0,4,100.000000,5,1
2,1,14,0,100.000000,0,0.000000,8.250000,0.150794,4,0.000000,...,8.142857,12,0,2,1,5828815,2,100.000000,5,1
3,0,62,3,62.500000,0,0.250000,5.000000,0.000000,2,0.000000,...,0.000000,1,0,0,0,107721,0,100.000000,2,0
4,0,224,6,0.000000,1,0.537037,6.333333,0.000000,2,0.000000,...,7.000000,6,0,4,2,8725,3,76.470588,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,0,448,6,0.000000,1,0.043478,7.750000,0.000000,2,0.000000,...,9.500000,4,0,8,0,3980,4,80.000000,4,0
11426,1,211,0,0.000000,1,0.000000,5.166667,0.023810,5,0.000000,...,4.900000,12,0,3,0,0,3,100.000000,5,1
11427,1,2809,10,80.000000,1,0.000000,6.153846,0.142857,2,0.000000,...,6.272727,13,0,1,6,8,5,6.250000,5,0
11428,0,85,4,0.000000,1,0.050000,12.500000,0.000000,2,0.000000,...,0.000000,2,0,0,0,2455493,3,16.666667,3,0
